In [1]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)



In [2]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    elif 'ubuntu' in os.environ.get('USER'):
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/'
    

## Data Loading and PCA

In [3]:
path = os.path.join(DATA_DIR, 'split_data')

#loading
xtrain = pd.read_pickle(os.path.join(path, 'train_features_sensor.pkl'))
xtest = pd.read_pickle(os.path.join(path, 'test_features_sensor_regression.pkl'))
ytrain = pd.read_pickle(os.path.join(path, 'train_labels_regression.pkl'))
ytest = pd.read_pickle(os.path.join(path, 'test_labels_regression.pkl'))


In [4]:
# dealing with NaNs
train_set = pd.concat([xtrain, ytrain], axis=1)
train_set = train_set.dropna()

labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum', 'WISC_VCI_Sum']
xtrain = train_set[train_set.columns.difference(labels_list)]
ytrain = train_set[labels_list]

In [5]:
xtrain = np.array(xtrain)
xtest = np.array(xtest)
ytrain = np.array(ytrain) 
ytest = np.array(ytest) 

In [6]:
age_gender = True
if age_gender:
    train_age_gender = xtrain[:,-2:]
    test_age_gender = xtest[:,-2:]
    xtrain = xtrain[:,:-2]
    xtest = xtest[:,:-2]

In [7]:
# scaling x
norm = preprocessing.MinMaxScaler().fit(xtrain)

# transform training data
xtrain = norm.transform(xtrain)
xtest = norm.transform(xtest)
print(xtrain.shape, xtest.shape)

print('Applying PCA...')
pca = PCA(.95) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)
print(xtrain.shape, xtest.shape)

(802, 4305) (268, 4305)
Applying PCA...
(802, 200) (268, 200)


In [8]:
if age_gender:
    xtrain = np.concatenate([xtrain, train_age_gender], axis = 1)
    xtest = np.concatenate([xtest, test_age_gender], axis = 1)

In [9]:
# scaling y
min_max_scaler = preprocessing.MinMaxScaler().fit(ytrain)
ytrain = min_max_scaler.transform(ytrain)
ytest = min_max_scaler.transform(ytest)

print(ytrain.shape, ytest.shape)

(802, 7) (268, 7)


In [10]:
ytrain.mean(axis=0)

array([0.39968475, 0.33121278, 0.61656041, 0.54500321, 0.26357717,
       0.45678772, 0.49548922])

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor


lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))


Mean squared error (dummy): 13.98
Median absolute error (dummy): 1.58
r2_score (dummy mean): -29.04
r2_score (dummy median): -27.03


In [12]:
xtrain.shape

(802, 202)

In [13]:
ytrain.shape

(802, 7)

In [14]:
n = 10

## MOR regressor with base SVR regressor

In [15]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils import resample

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
model = MultiOutputRegressor(svr)
# Train the regressor
model = model.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = model.predict(xtest)


mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (SVR) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (SVR): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))




Mean squared error (SVR): 14.50 [14.50, 14.50]
Mean abs error (SVR) 1.68 [1.68, 1.68]
R2 score (SVR): -26.47 [-26.47, -26.47]


In [16]:
max_depth = 30
# define model
model = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
# fit model
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

									



Mean squared error (RanFor): 14.47 [14.47, 14.47]
Mean abs error (RanFor) 1.67 [1.67, 1.67]
R2 score (RanFor): -24.89 [-24.89, -24.89]


In [17]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (LinReg): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (LinReg) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (LinReg): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

#print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
#																	ypred)))
#print("Mean abs error (LinReg): {:.2f}".format(mean_absolute_error(ytest,
#																	ypred)))																	
#print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))		

Mean squared error (LinReg): 14.45 [14.45, 14.45]
Mean abs error (LinReg) 1.67 [1.67, 1.67]
R2 score (LinReg): -25.61 [-25.61, -25.61]


In [18]:
max_depth = 30
model = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
model.fit(xtrain, ytrain)

mse = []
r = []
mae = []
for i in range(n):
# Generate predictions for testing data
    # Resample
	resampled_xtest, resampled_ytest = resample(xtest, ytest, replace=True, n_samples=len(ytest), random_state=7+i)
	ypred = model.predict(resampled_xtest)
	mse.append(mean_squared_error(resampled_ytest,ypred))
	r.append(model.score(resampled_xtest, resampled_ytest))
	mae.append(mean_absolute_error(resampled_ytest,ypred))

print("Mean squared error (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mse), np.percentile(mse, 5), np.percentile(mse, 95)))
print("Mean abs error (RanFor) {:.2f} [{:.2f}, {:.2f}]".format(np.mean(mae), np.percentile(mae, 5), np.percentile(mae, 95)))
print("R2 score (RanFor): {:.2f} [{:.2f}, {:.2f}]".format(np.mean(r), np.percentile(r, 5), np.percentile(r, 95)))

Mean squared error (RanFor): 14.47 [14.47, 14.47]
Mean abs error (RanFor) 1.67 [1.67, 1.67]
R2 score (RanFor): -24.83 [-24.83, -24.83]


In [19]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(5, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(4, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(1, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))

model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())

Epoch 1/10
321/321 [==============================] - 2s 4ms/step - loss: 0.2969 - val_loss: 0.1982
Epoch 2/10
321/321 [==============================] - 2s 6ms/step - loss: 0.1748 - val_loss: 0.1663
Epoch 3/10
321/321 [==============================] - 1s 5ms/step - loss: 0.1624 - val_loss: 0.1659
Epoch 4/10
321/321 [==============================] - 2s 7ms/step - loss: 0.1615 - val_loss: 0.1681
Epoch 5/10
321/321 [==============================] - 2s 5ms/step - loss: 0.1608 - val_loss: 0.1668
Epoch 6/10
321/321 [==============================] - 1s 4ms/step - loss: 0.1608 - val_loss: 0.1652
Epoch 7/10
321/321 [==============================] - 2s 5ms/step - loss: 0.1606 - val_loss: 0.1667
Epoch 8/10
321/321 [==============================] - 2s 7ms/step - loss: 0.1605 - val_loss: 0.1659
Epoch 9/10
321/321 [==============================] - 2s 6ms/step - loss: 0.1599 - val_loss: 0.1665
Epoch 10/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1593 - val_loss: 0.1669

In [20]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(7, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))


model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())

Epoch 1/10
321/321 [==============================] - 2s 3ms/step - loss: 0.5015 - val_loss: 0.2709
Epoch 2/10
321/321 [==============================] - 1s 2ms/step - loss: 0.2067 - val_loss: 0.1842
Epoch 3/10
321/321 [==============================] - 1s 2ms/step - loss: 0.1655 - val_loss: 0.1733
Epoch 4/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1609 - val_loss: 0.1735
Epoch 5/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1593 - val_loss: 0.1743
Epoch 6/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1592 - val_loss: 0.1732
Epoch 7/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1579 - val_loss: 0.1726
Epoch 8/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1571 - val_loss: 0.1700
Epoch 9/10
321/321 [==============================] - 1s 3ms/step - loss: 0.1564 - val_loss: 0.1722
Epoch 10/10
321/321 [==============================] - 1s 5ms/step - loss: 0.1558 - val_loss: 0.1700